In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
s0 = 1000
r = 0
T = 1 / 12
K = 1000
xi = 0.5
# symbolic variables
s, v, t, v0 = symbols('s v t v0')
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / sqrt(v0 * t)
D0 = 0.5 * (v - v0) * s * density(N)(d_p) / sqrt(v0 * t)
delta0 = diff(D0, s)
gamma0 = diff(delta0, s)
vega0 = diff(D0, v)

def delta(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p)

def gamma(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p) / (sigma * np.sqrt(T) * S)

def vega(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return S * N(d_p) * np.sqrt(T) / (2 * sigma)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

D = [D0]
for _ in range(1, 5):
  D.append(L(D[-1]))

In [2]:
Delta = [delta0]
Delta_func = []
for i in range(1, 5):
  Delta.append(diff(D[i], s))
for i in range(0, 5):
  Delta_func.append(lambdify([s, v, t, v0], Delta[i]))

Gamma = [gamma0]
Gamma_func = []
for i in range(1, 5):
  Gamma.append(diff(Delta[i], s))
for i in range(0, 5):
  Gamma_func.append(lambdify([s, v, t, v0], Gamma[i]))

Vega = [vega0]
Vega_func = []
for i in range(1, 5):
  Vega.append(diff(D[i], v))
for i in range(0, 5):
  Vega_func.append(lambdify([s, v, t, v0], Vega[i]))

In [3]:
%%time
V = np.arange(0.1, 1.2, 0.1)
Approx_delta = []
for v in V:
  delta0_approx = delta(s0, K, T, r, np.sqrt(v)) + T * Delta_func[0](s0, v, T, v)
  delta1_approx = delta0_approx + T**2 / 2 * Delta_func[1](s0, v, T, v)
  delta2_approx = delta1_approx + T**3 / 6 * Delta_func[2](s0, v, T, v)
  delta3_approx = delta2_approx + T**4 / 24 * Delta_func[3](s0, v, T, v)
  delta4_approx = delta3_approx + T**5 / 120 * Delta_func[4](s0, v, T, v)
  Approx_delta.append(100*delta4_approx)
print(Approx_delta)

[51.95337478987898, 52.66214621541805, 53.21934729861749, 53.693164518367844, 54.112330658145154, 54.49211273698567, 54.841764715820865, 55.167396182328275, 55.473290951365406, 55.76259021564746, 56.037678550311874]
Wall time: 4.98 s


In [4]:
%%time
Approx_gamma = []
for v in V:
  gamma0_approx = gamma(s0, K, T, r, np.sqrt(v)) + T * Gamma_func[0](s0, v, T, v)
  gamma1_approx = gamma0_approx + T**2 / 2 * Gamma_func[1](s0, v, T, v)
  gamma2_approx = gamma1_approx + T**3 / 6 * Gamma_func[2](s0, v, T, v)
  gamma3_approx = gamma2_approx + T**4 / 24 * Gamma_func[3](s0, v, T, v)
  gamma4_approx = gamma3_approx + T**5 / 120 * Gamma_func[4](s0, v, T, v)
  Approx_gamma.append(100*gamma4_approx)
print(Approx_gamma)


[0.44336411361271905, 0.3117950361341641, 0.25375138883843307, 0.219253014017635, 0.19574695595913116, 0.1784086823538222, 0.1649371233384017, 0.1540776807162305, 0.1450805047019765, 0.13746624159819, 0.13091226264521527]
Wall time: 9.91 s


In [5]:
%%time
Approx_vega = []
for v in V:
  vega0_approx = vega(s0, K, T, r, np.sqrt(v))
  vega1_approx = vega0_approx + T**2 / 2 * Vega_func[1](s0, v, T, v)
  vega2_approx = vega1_approx + T**3 / 6 * Vega_func[2](s0, v, T, v)
  vega3_approx = vega2_approx + T**4 / 24 * Vega_func[3](s0, v, T, v)
  vega4_approx = vega3_approx + T**5 / 120 * Vega_func[4](s0, v, T, v)
  Approx_vega.append(vega4_approx)
print(Approx_vega)

[181.17852597026194, 127.90098351585404, 104.34754715014405, 90.2924185254633, 80.68839805416238, 73.58997681471622, 68.06630905346111, 63.608527597650564, 59.91183260643862, 56.781029272446396, 54.08454789922963]
Wall time: 3.65 s


In [7]:
Delta_FFT = np.array([51.9512, 52.6614, 53.2189, 53.6929, 54.1121, 54.4920, 54.8416, 55.1673, 55.4732, 55.7625, 56.0376])
Delta_diff = (Approx_delta - Delta_FFT) / Delta_FFT * 100
Gamma_FFT = np.array([0.4241, 0.2944, 0.2542, 0.2245, 0.2012, 0.1831, 0.1686, 0.1568, 0.1468, 0.1387, 0.1318])
Gamma_diff = (Approx_gamma - Gamma_FFT) / Gamma_FFT * 100
Vega_FFT = np.array([180.6213, 127.8147, 104.3381, 90.3166, 80.7070, 73.6120, 68.0870, 63.9556, 60.0878, 56.8783, 54.1400])
Vega_diff = (Approx_vega - Vega_FFT) / Vega_FFT * 100

df = pd.DataFrame({
    "V": V, 
    "FFT %delta": Delta_FFT, 
    "Replicate %delta": np.round(Approx_delta, 4), 
    "Delta %diff": np.round(Delta_diff, 4),
    "FFT %gamma": Gamma_FFT,
    "Replicate %gamma": np.round(Approx_gamma, 4), 
    "Gamma %diff": np.round(Gamma_diff, 4),
    "FFT %vega": Vega_FFT,
    "Replicate %vega": np.round(Approx_vega, 4),
    "Vega %diff": np.round(Vega_diff, 4)
    })
print(df)
df.to_csv("Panel ABC2 (s=1000).csv")

      V  FFT %delta  Replicate %delta  Delta %diff  FFT %gamma  \
0   0.1     51.9512           51.9534       0.0042      0.4241   
1   0.2     52.6614           52.6621       0.0014      0.2944   
2   0.3     53.2189           53.2193       0.0008      0.2542   
3   0.4     53.6929           53.6932       0.0005      0.2245   
4   0.5     54.1121           54.1123       0.0004      0.2012   
5   0.6     54.4920           54.4921       0.0002      0.1831   
6   0.7     54.8416           54.8418       0.0003      0.1686   
7   0.8     55.1673           55.1674       0.0002      0.1568   
8   0.9     55.4732           55.4733       0.0002      0.1468   
9   1.0     55.7625           55.7626       0.0002      0.1387   
10  1.1     56.0376           56.0377       0.0001      0.1318   

    Replicate %gamma  Gamma %diff  FFT %vega  Replicate %vega  Vega %diff  
0             0.4434       4.5424   180.6213         181.1785      0.3085  
1             0.3118       5.9086   127.8147         12